In [ ]:
val pathToJar = System.getProperty("user.dir") + "/HistogramModel.jar"
val x = ammonite.ops.Path(pathToJar)
interp.load.cp(x)

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits._
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Histogram
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.HistogramImpl

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.1`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

plotly.JupyterScala.init()

In [ ]:
val histogram = HistogramImpl(Map("a" -> 3, "qwe" -> 5))

In [ ]:
val histogram = HistogramImpl.extract("a b c a")

In [ ]:
implicit class DrawableHistogram[E](hist: Histogram[E]) {
    def draw() = {
        val xs = hist.elementsPresent.toSeq.sortBy(_.toString)
        val ys = xs.map(e => hist(e).toDouble)
        Bar(xs.map(_.toString), ys).plot()
    }
}

import ru.hse.se.ba.danilin.paul.histogram_model.atomizers.Pixel
import java.io.File
import javax.imageio.ImageIO.read

def readImages(folder: File) = {
    val files = (folder.isDirectory match {
      case true => folder.listFiles().toList
      case false => List(folder)
    }).filterNot(_.getName.startsWith("."))
    val names = files.map(_.getName.split('.')(0))
    val hists = files
      .filterNot(_.isHidden)
      .map(f => read(f))
      .map(_.toHistogram)
    names.zip(hists)
}

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.queries.HistogramInput

In [ ]:
val imagesHists = readImages(new File("images")).toMap.mapValues(_.normalize)
val imagesAliases = imagesHists.mapValues( hist => HistogramInput(hist))

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.queries.SubhistogramInput

import ru.hse.se.ba.danilin.paul.histogram_model.atomizers._

import ru.hse.se.ba.danilin.paul.histogram_model.queries.Query

In [ ]:
val pixelsAliases = Map("blue" -> SubhistogramInput(Set(Blue)),
                       "red" -> SubhistogramInput(Set(Red, Yellow)),
                       "green" -> SubhistogramInput(Set(Green)),
                       "yellow" -> SubhistogramInput(Set(Yellow)),
                       "cyan" -> SubhistogramInput(Set(Cyan)),
                       "magenta" -> SubhistogramInput(Set(Magenta)))

In [ ]:
import ru.hse.se.ba.danilin.paul.histogram_model.queries.Input

implicit val aliases = (Query.standardAliases[Pixel] ++ imagesAliases ++ pixelsAliases).asInstanceOf[Map[String, Input[Pixel]]]

In [ ]:
imagesHists.keys

In [ ]:
imagesHists("tiger")

In [ ]:
val q = Query.fromString("red | yellow")(aliases)

In [ ]:
val sea = imagesHists("sea")

val res = q.execute(sea).left.get

res similar sea

In [ ]:
val tiger = imagesHists("tiger")
tiger.draw

In [ ]:
val res = q.execute(tiger).left.get
res.draw
tiger similar sea

In [ ]:
val query = Query.fromString("red|yellow")(aliases)
val result = query.execute(imagesHists("tiger")).left.get
result.draw

In [ ]:
val newQuery = Query(query.injectHistogram(imagesHists("sea")))
val result = newQuery.execute(imagesHists("tiger")).left.get
result.draw

In [ ]:
val result = imagesHists("sea") + imagesHists("tiger")
result.draw

In [ ]:
result(Blue)

In [ ]:
imagesHists("sea") similar imagesHists("tiger")

In [ ]:
import io.circe.syntax._
import ru.hse.se.ba.danilin.paul.histogram_model.histogram.Implicits._
val json = result.asJson
val writer = new PrintWriter(new File("test result.json"))
writer.write(json.spaces2)
writer.close()

In [ ]:
val writer = new PrintWriter(new File("test result.json"))
writer.write(imagesHists("sea") similar imagesHists("tiger"))
writer.close()